In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import joblib

In [ ]:
base_path = '/kaggle/input/fabric-defect-dataset/Data Set'
captured_path = os.path.join(base_path, 'captured')
hole_path = os.path.join(base_path, 'hole')
horizontal_path = os.path.join(base_path, 'horizontal')
vertical_path = os.path.join(base_path, 'verticle')


In [ ]:
def load_images_from_folder(folder_path):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(gray_img)
            if 'hole' in folder_path:
                labels.append('hole')
            elif 'horizontal' in folder_path:
                labels.append('horizontal')
            elif 'verticle' in folder_path:
                labels.append('vertical')
            else:
                labels.append('unknown')
    return images, labels

captured_images, captured_labels = load_images_from_folder(captured_path)
hole_images, hole_labels = load_images_from_folder(hole_path)
horizontal_images, horizontal_labels = load_images_from_folder(horizontal_path)
vertical_images, vertical_labels = load_images_from_folder(vertical_path)


In [ ]:
radius = 2
n_points = 8 * radius

def apply_lbp(image):
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    return lbp

def apply_lbp_to_images(images):
    lbp_images = [apply_lbp(image) for image in images]
    return lbp_images

lbp_captured_images = apply_lbp_to_images(captured_images)
lbp_hole_images = apply_lbp_to_images(hole_images)
lbp_horizontal_images = apply_lbp_to_images(horizontal_images)
lbp_vertical_images = apply_lbp_to_images(vertical_images)


In [ ]:

def calculate_lbp_histogram(lbp_image, bins=256):
    hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, bins + 1), density=True)
    return hist

def calculate_histograms(lbp_images):
    histograms = [calculate_lbp_histogram(lbp) for lbp in lbp_images]
    return histograms
    
histograms_captured = calculate_histograms(lbp_captured_images)
histograms_hole = calculate_histograms(lbp_hole_images)
histograms_horizontal = calculate_histograms(lbp_horizontal_images)
histograms_vertical = calculate_histograms(lbp_vertical_images)


In [ ]:
X = np.array(histograms_hole + histograms_horizontal + histograms_vertical)
y = np.array(hole_labels + horizontal_labels + vertical_labels)


param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib
import numpy as np

svc = SVC(kernel='rbf', probability=True)

splits = {
    "70-30": 0.3,
    "80-20": 0.2,
    "90-10": 0.1
}

for split_name, test_size in splits.items():
    print(f"\nEvaluating with {split_name} train-test split")
    
    accuracies = []
    
    for i in range(10):
        print(f"\nRun {i+1} for {split_name} split")
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)
        grid_search = GridSearchCV(svc, param_grid, refit=True, verbose=2, cv=5)
        grid_search.fit(X_train, y_train)
        
        best_params = grid_search.best_params_
        best_model = grid_search.best_estimator_

        print(f"Best Parameters found for run {i+1}: {best_params}")
        
        model_filename = f'best_fabric_defect_model_{split_name}_run{i+1}.pkl'
        joblib.dump(best_model, model_filename)
        print(f"Model saved as {model_filename}")
        
        y_pred = best_model.predict(X_test)
        
        print(f"Classification report for run {i+1}:")
        print(classification_report(y_test, y_pred))
        
        accuracy = accuracy_score(y_test, y_pred) * 100
        accuracies.append(accuracy)
        print(f"Accuracy for run {i+1}: {accuracy:.2f}%")
    
    avg_accuracy = np.mean(accuracies)
    print(f"\nAverage Accuracy for {split_name} over 10 runs: {avg_accuracy:.2f}%")


In [ ]:
import random
def visualize_lbp(original_images, lbp_images, title):
    plt.figure(figsize=(12, 6))
    indices = random.sample(range(len(original_images)), 5)
    for i, idx in enumerate(indices):
        plt.subplot(2, 5, i+1)
        plt.imshow(original_images[idx], cmap='gray')
        plt.title(f"Original - {title}")
        plt.axis('off')
        plt.subplot(2, 5, i+6)
        plt.imshow(lbp_images[idx], cmap='gray')
        plt.title(f"LBP - {title}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_lbp(hole_images, lbp_hole_images, 'Hole')
visualize_lbp(horizontal_images, lbp_horizontal_images, 'Horizontal')
visualize_lbp(vertical_images, lbp_vertical_images, 'Vertical')


In [ ]:
classifier = joblib.load('best_fabric_defect_model_80-20_run3.pkl')


In [ ]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import joblib

def preprocess_image_from_path(image_path, radius=3, n_points=8 * 3):
    img = load_image_from_path(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    hist = calculate_lbp_histogram(lbp)

    return hist, gray, lbp, img

    
def load_image_from_path(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    return img
    
def calculate_lbp_histogram(lbp_image, bins=256):
    hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, bins + 1), density=True)
    return hist
    
def find_defect_center(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5, 5), 0)
    _, thresh = cv2.threshold(
        blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    if not contours:
        return None, 0

    largest = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(largest)

    M = cv2.moments(largest)
    if M["m00"] == 0:
        return None, area

    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    return (cx, cy), area
    
def estimate_severity(defect_area, image_area):
    ratio = defect_area / image_area

    if ratio < 0.01:
        return "Low"
    elif ratio < 0.05:
        return "Medium"
    else:
        return "High"

def predict_defect_from_path(image_path, classifier):
    hist, gray, lbp, img = preprocess_image_from_path(image_path)
    hist = hist.reshape(1, -1)

    pred_class = classifier.predict(hist)[0]

    if hasattr(classifier, "predict_proba"):
        probabilities = classifier.predict_proba(hist)[0]
    else:
        scores = classifier.decision_function(hist)
        probs = np.exp(scores - np.max(scores))
        probabilities = probs / probs.sum()

    class_confidence = {
        cls: round(float(prob), 4)
        for cls, prob in zip(classifier.classes_, probabilities)
    }

    center, defect_area = find_defect_center(gray)
    severity = estimate_severity(
        defect_area, gray.shape[0] * gray.shape[1]
    )

    return {
        "defect_type": pred_class,
        "confidence_scores": class_confidence,
        "defect_center_xy": center,
        "severity": severity
    }

classifier = joblib.load('best_fabric_defect_model_80-20_run3.pkl')

# add your image path here
IMAGE_PATH = "/kaggle/input/hole-image/hole_image.jpg"

result = predict_defect_from_path(IMAGE_PATH, classifier)

print("Defect Type      :", result["defect_type"])
print("Confidence Scores:", result["confidence_scores"])
print("Defect Center    :", result["defect_center_xy"])
print("Severity         :", result["severity"])

